# Recommending Users whom you should follow based on your tweet/profiles

Importing required Libraries

In [45]:
import pandas as pd
import seaborn as sns
%matplotlib inline
import numpy as np
from gensim.models import Word2Vec
import warnings
warnings.filterwarnings('ignore')
from  nltk.tokenize  import  TweetTokenizer 
import glob

Loading data into Dataframes in Pandas

In [46]:
allFiles = glob.glob("Data/*/*.csv", recursive=True)
frame = pd.DataFrame()

filenames = []
for file_ in allFiles:    
    df = pd.read_csv(file_, encoding = "ISO-8859-1")
    filenames.append(file_.split("\\")[-1:][0].split(".")[0])
    frame = pd.concat([frame,df[df.columns[1]]])


Print a sample. 5 from The dataframe.

In [47]:
print(frame.head())

                                                   0
0  b'RT @wyomingpd: Undercover #420day operations...
1  b"RT @HuffPost: On #420day, here's the exhaust...
2  b'@Bevmedium @Bri_Clark Love this! #truth #fre...
3  b'RT @deadpoolmovie: Happy #420Day everyone! L...
4  b' #420day RT @TrevDon: Happy Birthday Dogbert...


Tokenize the tweets for all hashtags in the dataframe

In [48]:
corpus = []
tknzr  =  TweetTokenizer () 

for row in frame[frame.columns[0]]:
    corpus.append(tknzr . tokenize (row))

In [49]:
print (corpus[:10])

[["b'RT", '@wyomingpd', ':', 'Undercover', '#420day', 'operations', 'are', 'in', 'place', '.', '\\', 'n', '\\', 'nIncognito', 'traps', 'have', 'been', 'set', 'up', 'throughout', 'the', 'city', '.', '\\', 'n', '\\', 'n', '#Happy420', 'https://t.co\\xe2\\x80\\xa6', "'"], ['b', '"', 'RT', '@HuffPost', ':', 'On', '#420day', ',', "here's", 'the', 'exhaustive', 'list', 'of', 'everyone', "who's", 'died', 'of', 'a', 'marijuana', 'overdose', '.', '\\', 'n', '\\', 'nOh', 'wait', '.', '\\', 'n', '\\', "nThere's", 'no', 'one', '.', 'https://t.\\xe2\\x80\\xa6', '"'], ['b', "'", '@Bevmedium', '@Bri_Clark', 'Love', 'this', '!', '#truth', '#freespirted', '#420day', "'"], ["b'RT", '@deadpoolmovie', ':', 'Happy', '#420Day', 'everyone', '!', 'Let', '\\', 'xe2', '\\', 'x80', '\\', 'x99s', 'be', 'careful', 'out', 'there', '!', 'https://t.co/qLLDEU7xQO', "'"], ['b', "'", '#420day', 'RT', '@TrevDon', ':', 'Happy', 'Birthday', 'Dogbert', '!', 'The', 'big', '8', '.', 'Even', 'though', 'each', 'day', 'with', 'y

 Import Spacy. Spacy is used for vectorization of text and finding similarity between texts in corpus using cosine similarity and part-of-speech tagging.

In [59]:
import spacy

spacy.load("en")
import en_core_web_sm
nlp = en_core_web_sm.load()

location_df = pd.DataFrame()
locationFiles = glob.glob("Data/*/*.csv", recursive=True)

frames = []
frame_words = []

for file in locationFiles:
    df = pd.read_csv(file, encoding = "ISO-8859-1")
    location_df = pd.concat([location_df,df[df.columns[1]]])

for index,frame in location_df.iterrows():
    frames.append(nlp(frame[0]))
    frame_words.append(frame[0])


Run our Spacy model to predict the output for an input sentence. We use a real example tweet below for testing. Can be changed with any other tweet.

In [60]:
import operator

input = "Self employment is best!! I wnt my dghtr to learn 2 be a role model!"


user_input_nlp = nlp(input)
mostsimilarword = ""
mostsimilarword_similarity = 0
reference = {}
index = 0

for idx, doc in enumerate(frames):
    similarity_of_word = doc.similarity(user_input_nlp)
    if (similarity_of_word > mostsimilarword_similarity):
        R
        print(doc)
        similar_tweet = frame_words[idx]
        if "@" in similar_tweet:
            mostsimilarword_similarity = similarity_of_word
            mostsimilarword = similar_tweet
            reference = set([i[1:].rstrip(':') for i in similar_tweet.split() if i.startswith("@")])

print("Similar Tweet: " + mostsimilarword)

top_reference = reference.pop()

print("\n You might like to befriend: ")
print("@"+top_reference)


b'RT @wyomingpd: Undercover #420day operations are in place. \n\nIncognito traps have been set up throughout the city. \n\n#Happy420 https://t.co\xe2\x80\xa6'
b"RT @HuffPost: On #420day, here's the exhaustive list of everyone who's died of a marijuana overdose.\n\nOh wait. \n\nThere's no one. https://t.\xe2\x80\xa6"
b' #420day RT @TrevDon: Happy Birthday Dogbert!  The big 8.  Even though each day with your   dog   is a day to be celebrated, today is extra special.\xe2\x80\xa6'
b" #420day RT @MrFilmkritik: Today is 4/20, so here\xe2\x80\x99s a reminder that if y'all support the legalization of marijuana, you also need to be supporting d\xe2\x80\xa6"
b'RT @SomeonesAnIdiot: One of the greatest quotes of all time #420day https://t.co/tAarClAaS2'
b' #420day RT @GrowBigBuds: I have gathered and carefully selected all the information on my website for years. I believe that is one of the best reso\xe2\x80\xa6'
b'I forgot it was #420day because every day is 420 for me! https://t.co/zRgDUH9vHs'


Inport and Run Tweepy API to access the details of the above recommended profile.

In [52]:
import tweepy

In [61]:
consumer_key = 'xE5rQKgl43y7hsYEHVK6xfPhD'
consumer_secret = 'UZ0YELUFQRLrI08kM2jS4EOCVzp4kN2H4XbmyZptxHSt7Gjbm1'
access_token = '971850294962393089-HiM8wvFzLZQ9hRlLCtMEbsQ3CJNoirS'
access_token_secret = 'KOAm02mlcwTIklMWBr4VNpjKVrsKrEP3QISS3uhiz0mfZ'

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth,wait_on_rate_limit=True)

Access the Status of the user and his timeline.

In [62]:
stuff = api.user_timeline(screen_name = top_reference, count = 100, include_rts = True, exclude_replies=True)

print (stuff)




[Status(_api=<tweepy.api.API object at 0x0000015AA2CD1400>, _json={'created_at': 'Sat Apr 21 19:02:57 +0000 2018', 'id': 987768671312470016, 'id_str': '987768671312470016', 'text': 'Can’t wait for Monday’s surprise ...  👟', 'truncated': False, 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [], 'urls': []}, 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>', 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 5768872, 'id_str': '5768872', 'name': 'Gary Vaynerchuk', 'screen_name': 'garyvee', 'location': 'NYC', 'description': 'Family 1st! Entrepreneur 2nd. CEO of @Vaynermedia and @VaynerSports. Investor in Twitter, Snapchat, Uber, Venmo. 5X NYTimes best 📚. Die hard NYJets fan', 'url': 'https://t.co/I1Xzbq4vX9', 'entities': {'url': {'urls': [{'url': 'https://t.co/I1Xzbq4vX9', 'expanded_url': 'https://www.garyva

Retreive the user name, location and status along with the users banner image to display below.

In [63]:
#for status in stuff:
status = stuff[0]
print ("User Name:" + status.user.name)
print ("User Status:" + status.user.description)
if(status.user.location): 
    print ("User Location:" + status.user.location)
    
print (status.user.profile_banner_url)


User Name:Gary Vaynerchuk
User Status:Family 1st! Entrepreneur 2nd. CEO of @Vaynermedia and @VaynerSports. Investor in Twitter, Snapchat, Uber, Venmo. 5X NYTimes best 📚. Die hard NYJets fan
User Location:NYC
https://pbs.twimg.com/profile_banners/5768872/1519958066


Display the profile image.

In [64]:
from IPython.display import Image
from IPython.core.display import HTML 
Image(url= status.user.profile_banner_url)


License added below:

The MIT License (MIT)

Copyright (c) 2016-2017 Rohan Jahagirdar, Sai Nikhil Dogiparty

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.